In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
#import matplotlib.pyplot as plt
import gzip
import json
#from tabulate import tabulate
import bz2
#import os
from tqdm import tqdm

# Load the Data

## Helper functions and variables

In [27]:
def delta_date(x):
    return (str_to_date(x['end_date']) - str_to_date(x['start_date'])).days + 1

In [28]:
def str_to_date(d):
    return datetime.strptime(d, "%Y-%m-%d")

In [29]:
datapath="irrs/"

In [31]:
def overlap_db(db):
    timeless_overlap = db.merge(bgp, left_on='route', right_on='prefix', suffixes=('_route', '_prefix'))
    timeless_overlap['end_date'] = pd.to_datetime(timeless_overlap['end_date'], format="%Y-%m-%d")
    timeless_overlap['start_date'] = pd.to_datetime(timeless_overlap['start_date'], format="%Y-%m-%d")
    timeless_overlap['start']=pd.to_datetime(timeless_overlap['start'], unit='s').dt.strftime('%Y-%m-%d')
    timeless_overlap['end']=pd.to_datetime(timeless_overlap['end'], unit='s').dt.strftime('%Y-%m-%d')
    #timeless_overlap['origin']=timeless_overlap['origin'].str.replace('AS',"")
    actual_overlap = timeless_overlap[(timeless_overlap.end_date > timeless_overlap.start)&(timeless_overlap.start_date < timeless_overlap.end)]
    actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)
    return actual_overlap

## Load BGP

In [33]:
bgp = pd.read_csv('pfx2as_2021_2023_merged.csv.gz', names=['prefix', 'asn', 'start', 'end'], delimiter=' ',low_memory=False)
bgp['duration'] = bgp['end'] - bgp['start']
bgpagg = bgp.groupby(['prefix', 'asn']).agg({'duration':'sum'}).reset_index()
bgpagg['asn']='AS'+bgpagg['asn'].astype(str)

## Load Altdb

In [34]:
altdb = pd.read_json(datapath+'altdb/altdb.route.json.gz', lines=True)
altdb['lifetime'] = altdb.apply(delta_date, axis=1)
altdb_overlap=overlap_db(altdb)
altdbagg = altdb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

<ipython-input-31-bca60fb6450b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)


## Load Radb

In [400]:
radb = pd.read_json(datapath+'radb/radb.route.json.gz', lines=True)
radb['lifetime'] = radb.apply(delta_date, axis=1)
radb_overlap=overlap_db(radb)
radbagg = radb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

<ipython-input-399-bca60fb6450b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)


## Load Afrinic

In [288]:
afrinic = pd.read_json(datapath+'afrinic/afrinic.route.json.gz', lines=True)
afrinic['lifetime'] = afrinic.apply(delta_date, axis=1)
afrinic_overlap=overlap_db(afrinic)
afrinicagg = afrinic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Jpirr

In [289]:
jpirr = pd.read_json(datapath+'jpirr/jpirr.route.json.gz', lines=True)
jpirr['lifetime'] = jpirr.apply(delta_date, axis=1)
jpirr_overlap=overlap_db(jpirr)
jpirragg = jpirr_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Canarie

In [290]:
canarie = pd.read_json(datapath+'canarie/canarie.route.json.gz', lines=True)
canarie['lifetime'] = canarie.apply(delta_date, axis=1)
canarie_overlap=overlap_db(canarie)
canarieagg = canarie_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Apnic

In [291]:
apnic = pd.read_json(datapath+'apnic/apnic.route.json.gz', lines=True)
apnic['lifetime'] = apnic.apply(delta_date, axis=1)
apnic_overlap=overlap_db(apnic)
apnicagg = apnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Arin-Nonauth

In [292]:
arin_nonauth = pd.read_json(datapath+'arin-nonauth/arin-nonauth.route.json.gz', lines=True)
arin_nonauth['lifetime'] = arin_nonauth.apply(delta_date, axis=1)
arin_nonauth_overlap=overlap_db(arin_nonauth)
arin_nonauthagg = arin_nonauth_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Level3

In [293]:
level3 = pd.read_json(datapath+'level3/level3.route.json.gz', lines=True)
level3['lifetime'] = level3.apply(delta_date, axis=1)
level3_overlap=overlap_db(level3)
level3agg = level3_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Nestegg

In [294]:
nestegg = pd.read_json(datapath+'nestegg/nestegg.route.json.gz', lines=True)
nestegg['lifetime'] = nestegg.apply(delta_date, axis=1)
nestegg_overlap=overlap_db(nestegg)
nesteggagg = nestegg_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load BBoi

In [295]:
bboi= pd.read_json(datapath+'bboi/bboi.route.json.gz', lines=True)
bboi['lifetime'] = bboi.apply(delta_date, axis=1)
bboi_overlap=overlap_db(bboi)
bboiagg = bboi_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load idnic

In [296]:
idnic= pd.read_json(datapath+'idnic/idnic.route.json.gz', lines=True)
idnic['lifetime'] = idnic.apply(delta_date, axis=1)
idnic_overlap=overlap_db(idnic)
idnicagg = idnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load wcgdb

In [297]:
wcgdb= pd.read_json(datapath+'wcgdb/wcgdb.route.json.gz', lines=True)
wcgdb['lifetime'] = wcgdb.apply(delta_date, axis=1)
wcgdb_overlap=overlap_db(wcgdb)
wcgdbagg = wcgdb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load RGNET

In [298]:
rgnet= pd.read_json(datapath+'rgnet/rgnet.route.json.gz', lines=True)
rgnet['lifetime'] = rgnet.apply(delta_date, axis=1)
rgnet_overlap=overlap_db(rgnet)
rgnetagg = rgnet_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load tc

In [299]:
tc= pd.read_json(datapath+'tc/tc.route.json.gz', lines=True)
tc['lifetime'] = tc.apply(delta_date, axis=1)
tc_overlap=overlap_db(tc)
tcagg = tc_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Lacnic

In [300]:
lacnic= pd.read_json(datapath+'lacnic/lacnic.route.json.gz', lines=True)
lacnic['lifetime'] = lacnic.apply(delta_date, axis=1)
lacnic_overlap=overlap_db(lacnic)
lacnicagg = lacnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Ripe-nonauth

In [301]:
ripe_nonauth= pd.read_json(datapath+'ripe-nonauth/ripe-nonauth.route.json.gz', lines=True)
ripe_nonauth['lifetime'] = ripe_nonauth.apply(delta_date, axis=1)
ripe_nonauth_overlap=overlap_db(ripe_nonauth)
ripe_nonauthagg = ripe_nonauth_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Openface

In [302]:
openface= pd.read_json(datapath+'openface/openface.route.json.gz', lines=True)
openface['lifetime'] = openface.apply(delta_date, axis=1)
openface_overlap=overlap_db(openface)
openfaceagg = openface_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Panix

In [303]:
panix= pd.read_json(datapath+'panix/panix.route.json.gz', lines=True)
panix['lifetime'] = panix.apply(delta_date, axis=1)
panix_overlap=overlap_db(panix)
panixagg = panix_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Arin

In [304]:
arin= pd.read_json(datapath+'arin/arin.route.json.gz', lines=True)
arin['lifetime'] = arin.apply(delta_date, axis=1)
arin_overlap=overlap_db(arin)
arinagg = arin_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Ripe

In [305]:
ripe= pd.read_json(datapath+'ripe/ripe.route.json.gz', lines=True)
ripe['lifetime'] = ripe.apply(delta_date, axis=1)
ripe_overlap=overlap_db(ripe)
ripeagg = ripe_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Nttcom

In [306]:
nttcom= pd.read_json(datapath+'nttcom/nttcom.route.json.gz', lines=True)
nttcom['lifetime'] = nttcom.apply(delta_date, axis=1)
nttcom_overlap=overlap_db(nttcom)
nttcomagg = nttcom_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

# Table 2: Calculate number of prefix-origin pairs in every IRR, overlap of prefix-origin pairs with BGP and all IRRs, & generate table

## Helper functions

In [307]:
def numerator_overlap(dfagg):
    overlap=len(dfagg.merge(aggbgp, left_on=['route', 'origin'], right_on=['prefix', 'asn']))
    return overlap

In [308]:
def denominator(df):
    return len(df)

In [313]:
def percentage_frac(dfagg,df):
    num=numerator_overlap(dfagg)
    denom=denominator(df)
    return str(round((num*100/denom),2))+"\% ("+str(num)+"/"+str(denom)+")"

In [314]:
print("\\begin{table}[!h]")
print("\\begin{tabular}{l|l|l}")
print("\\toprule")
print("IRR & \# Prefix-Origin Pairs & \% Prefix-Origin Pairs in BGP "+chr(92)+chr(92))
print("\\midrule")
print("AFRINIC","&",denominator(afrinic),"&", percentage_frac(afrinicagg,afrinic), chr(92)+chr(92))
print("\\midrule")
print("ALTDB","&",denominator(altdb),"&", percentage_frac(altdbagg,altdb), chr(92)+chr(92))
print("\\midrule")
print("APNIC","&",denominator(apnic),"&", percentage_frac(apnicagg,apnic), chr(92)+chr(92))
print("\\midrule")
print("ARIN","&",denominator(arin),"&", percentage_frac(arinagg,arin), chr(92)+chr(92))
print("\\midrule")
print("ARIN-NA","&",denominator(arin_nonauth),"&", percentage_frac(arin_nonauthagg,arin_nonauth), chr(92)+chr(92))
print("\\midrule")
print("BBOI","&",denominator(bboi),"&", percentage_frac(bboiagg,bboi), chr(92)+chr(92))
print("\\midrule")
print("CANARIE","&",denominator(canarie),"&", percentage_frac(canarieagg,canarie), chr(92)+chr(92))
print("\\midrule")
print("IDNIC","&",denominator(idnic),"&", percentage_frac(idnicagg,idnic), chr(92)+chr(92))
print("\\midrule")
print("JPIRR","&",denominator(jpirr),"&", percentage_frac(jpirragg,jpirr), chr(92)+chr(92))
print("\\midrule")
print("LACNIC","&",denominator(lacnic),"&", percentage_frac(lacnicagg,lacnic), chr(92)+chr(92))
print("\\midrule")
print("LEVEL3","&",denominator(level3),"&", percentage_frac(level3agg,level3), chr(92)+chr(92))
print("\\midrule")
print("NESTEGG","&",denominator(nestegg),"&", percentage_frac(nesteggagg,nestegg), chr(92)+chr(92))
print("\\midrule")
print("NTTCOM","&",denominator(nttcom),"&", percentage_frac(nttcomagg,nttcom), chr(92)+chr(92))
print("\\midrule")
print("OPENFACE","&",denominator(openface),"&", percentage_frac(openfaceagg,openface), chr(92)+chr(92))
print("\\midrule")
print("PANIX","&",denominator(panix),"&", percentage_frac(panixagg,panix), chr(92)+chr(92))
print("\\midrule")
print("RADB","&",denominator(radb),"&", percentage_frac(radbagg,radb), chr(92)+chr(92))
print("\\midrule")
print("RGNET","&",denominator(rgnet),"&", percentage_frac(rgnetagg,rgnet), chr(92)+chr(92))
print("\\midrule")
print("RIPE","&",denominator(ripe),"&", percentage_frac(ripeagg,ripe), chr(92)+chr(92))
print("\\midrule")
print("RIPE-NA","&",denominator(ripe_nonauth),"&", percentage_frac(ripe_nonauthagg,ripe_nonauth), chr(92)+chr(92))
print("\\midrule")
print("TC","&",denominator(tc),"&", percentage_frac(tcagg,tc), chr(92)+chr(92))
print("\\midrule")
print("WCGDB","&",denominator(wcgdb),"&", percentage_frac(wcgdbagg,wcgdb), chr(92)+chr(92))
print("\\midrule")
print("\\bottomrule")
print("\\end{tabular}")
print("\\caption{insert}")
print("\\end{table}")

\begin{table}[!h]
\begin{tabular}{l|l|l}
\toprule
IRR & \# Prefix-Origin Pairs & \% Prefix-Origin Pairs in BGP \\
\midrule
AFRINIC & 106842 & 20.37\% (21759/106842) \\
\midrule
ALTDB & 27493 & 58.35\% (16043/27493) \\
\midrule
APNIC & 701196 & 17.32\% (121480/701196) \\
\midrule
ARIN & 104298 & 49.35\% (51467/104298) \\
\midrule
ARIN-NA & 66126 & 18.4\% (12169/66126) \\
\midrule
BBOI & 952 & 51.89\% (494/952) \\
\midrule
CANARIE & 1460 & 58.42\% (853/1460) \\
\midrule
IDNIC & 6158 & 64.44\% (3968/6158) \\
\midrule
JPIRR & 42546 & 21.42\% (9114/42546) \\
\midrule
LACNIC & 97908 & 10.01\% (9800/97908) \\
\midrule
LEVEL3 & 121534 & 18.47\% (22452/121534) \\
\midrule
NESTEGG & 4 & 75.0\% (3/4) \\
\midrule
NTTCOM & 502051 & 13.77\% (69146/502051) \\
\midrule
OPENFACE & 17 & 41.18\% (7/17) \\
\midrule
PANIX & 40 & 15.0\% (6/40) \\
\midrule
RADB & 1875251 & 23.7\% (444479/1875251) \\
\midrule
RGNET & 44 & 47.73\% (21/44) \\
\midrule
RIPE & 469651 & 56.43\% (265001/469651) \\
\midrule
RIPE-NA 